### Reference Notbooks
4. Natural language Processing IV (Section 5.4 thne 5.6)
4. NLP Exercise - do with student

### Steps to Follow
1. Try NaiveBayes classiffier first
2. Data Cleaning, lemming, etc
3. TFIDF
5. Try Random Forest (or K-means with only 2 clases?)

In [19]:
#%pip install spacy
#%pip install blis


#sample change

Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
#import spacy
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Javi\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Javi\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Javi\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\Javi\anaconda3\Lib\site-packages

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.




A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "c:\Users\Javi\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>
    app.launch_new_instance()
  File "c:\Users\Javi\anaconda3\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "c:\Users\Javi\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start
    self.io_loop.start()
  File "c:\Users\Javi\anaconda3\Lib\site-packages

ImportError: 
A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.2 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.



ImportError: numpy.core.multiarray failed to import

In [12]:
# Importing the Training Data file
df= pd.read_csv('TRAINING_DATA.txt', sep='\t')
df.columns = ['tag', 'text']
df.sample(10)

,tag,text
8337,1,"En muchos sentidos , este es sólo otro ejemplo..."
3246,1,James Snyder aporta mucho entusiasmo a su rol ...
12976,1,"Etiquetas : anglofilia , detenida , británica ..."
4484,1,Me gustaría animar a todos que le den un vista...
5028,1,"Al ser bueno con tus manos , dijo , aseguras t..."
2136,0,No hay una hora del día en que usted no puede ...
9939,1,Los niños que se ven privados de la interacció...
2972,1,Yo soy de la opinión más optimista .
5536,0,Estamos de acuerdo en que cada padre tiene el ...
10240,0,Le sacas mucho más fuera de las sesiones .


In [ ]:
def clean_data(text):        
    text = ''.join(f' {e} ' if e in string.punctuation else e for e in text)  # Add spaces around special characters
    text = ''.join(e for e in text if e.isalpha() or e.isspace())  # Remove special characters and numbers
    text = ' '.join(word for word in text.split() if len(word) > 1)  # Remove single characters
    text = text.lstrip()  # Remove single characters from the start
    text = ' '.join(text.split())  # Substitute multiple spaces with single space
    text = text.replace('b ', '')  # Remove prefixed 'b'
    text = text.lower()  # Convert to lowercase
    text = text.strip()  # Remove leading/trailing whitespace
    
    return text

clean_df = df.apply(clean_data)
clean_df.sample(10)

In [ ]:
# Get the Spanish stopwords
stop_words = set(stopwords.words('spanish'))
# Remove stopwords
no_sw_df = clean_df.apply(lambda x: ' '.join(word for word in x.split() if word not in stop_words))

In [ ]:
### TODO: Lemmatization

In [ ]:
### TODO: Bag of Words?

In [13]:
#Defining target and features
X = df['text']
y = df['tag']

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
### TODO: Bag of Words or some way of vectorizing the data
countvector=CountVectorizer(ngram_range=(2,2))
traindataset=countvector.fit_transform(headlines)

In [ ]:
### Testing Naive Bayes out of the box
#Creating and Training the Naive bayes model
nb_model = MultinomialNB().fit(X_train, y_train)

In [ ]:
### Making predictions and Evaluating the model
## Predict for the Test Dataset
y_pred = nb_model.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test, y_pred)
print(matrix)

acc_score = accuracy_score(X_test, y_pred)
print(f"Accuracy Score: {acc_score:.4f}")

report = classification_report(X_test, y_pred)
print(report)

In [ ]:
""" # Creating a confusion matrix
cm = confusion_matrix(
    np.argmax(y_test, axis=1),                                                 # True class labels converted from one-hot encoding to class indices
    np.argmax(y_pred, axis=1)                                                  # Predicted class labels converted from one-hot encoding to class indices
)

# Plot the confusion matrix
plt.figure(figsize=(8, 6))                                                     #   Set figure size to make the plot readable
plt.imshow(cm, interpolation='nearest', cmap='Greens')                         # Display the confusion matrix with a green color map
plt.title('Confusion Matrix')                                                  # Set the title of the plot
plt.colorbar()                                                                 # Add a color bar to interpret the matrix values visually

# Use the custom class names as tick labels
plt.xticks(range(len(custom_class_names)), custom_class_names, rotation=45)    # Set x-axis tick marks and labels with a 45-degree rotation
plt.yticks(range(len(custom_class_names)), custom_class_names)                 # Set y-axis tick marks and labels

# Add numbers to the confusion matrix
for i in range(cm.shape[0]):                                                   # Iterate over rows
    for j in range(cm.shape[1]):                                               # Iterate over columns
        plt.text(j, i, cm[i, j], ha='center', va='center', color='black')      # Add the count in each cell with black text

plt.show()  # Display the confusion matrix plot """